In [3]:
from jira import JIRA
import pandas as pd

#%load_ext lab_black

# How Frequently an OBS ticket appears?

## Helper Functions



In [1]:
def fetch_issues_with_details(email, api_token, server, project_key, start_date, end_date):
    """
    Fetches issues from JIRA project with all required details for ticket that were updated
    in a specified time frame
    .

    Parameters:
    email (str): The email address of the user.
    api_token (str): The API token for authentication.
    server (str): The URL of the JIRA server.
    project_key (str): The key of the project in JIRA.
    start_date (str): The start date of the time frame in the format 'YYYY-MM-DD'.
    end_date (str): The end date of the time frame in the format 'YYYY-MM-DD'.

    Returns:
    pandas.DataFrame: A DataFrame containing the fetched issues with their details.
    """
    
    jira = JIRA(basic_auth=(email, api_token), options={'server': server})
    
    # Query for issues updated in the specified time frame
    jql_query = f"project = {project_key} AND updated >= '{start_date}' AND updated <= '{end_date}'"
    
    # Check the total number of issues to be fetched
    total_issues = jira.search_issues(jql_query, maxResults=0).total
    print(f"Total issues to be fetched: {total_issues}")

    if list_only:
        return total_issues  # Exit the function, returning the total count

    # Fetch issues with all required details
    issues = jira.search_issues(jql_query, expand='renderedFields, changelog', maxResults=False)

    issues_data = []
    for issue in issues:
        issues_data.append({
            'key': issue.key,
            'summary': issue.fields.summary,
            'created': issue.fields.created,
            'updated': issue.fields.updated,
            'assignee': issue.fields.assignee.displayName if issue.fields.assignee else 'Unassigned',
            'reporter': issue.fields.reporter.displayName if issue.fields.reporter else 'Unknown',
            'priority': issue.fields.priority.name if issue.fields.priority else 'None',
            'status': issue.fields.status.name,
            'resolution': issue.fields.resolution.name if issue.fields.resolution else 'Unresolved',
            'comments': [{'author': comment.author.displayName, 'created': comment.created, 'body': comment.body} for comment in issue.fields.comment.comments],
            'histories': [{'author': history.author.displayName, 'created': history.created, 'items': history.items} for history in issue.changelog.histories]
        })
    
    return pd.DataFrame(issues_data)

In [3]:
email = 'dsanmartim@lsst.org'
api_token = 'ATATT3xFfGF0aDZ_zqSdqL_Zmd6Z7skvMZkwM6cj-tijPw5ankdXQ0ItXUKfuIkzfOqErBGtLNM_C0LI2Hg0w-utZC5yEH_-UK8KE5m-nVMqu7guXrhqAtVXNBnONPyunvHv37-S98ZeH99jp1Ze23JJFFOiXuA9mWI9N-nBcNuP0jJXE_B4_gY=4FCBB2F0'
server = 'https://rubinobs.atlassian.net/'
project_key = 'OBS'
start_date = '2024-03-18'
end_date = '2024-03-25'
